# **01_data_parsing.ipynb**

В этом ноутбуке собираются данные о недвижимости Москвы.

Данные получены с помощью парсинга с сайта Move.ru.

Результат ноутбука — таблица с сырыми объектами (цены, площади, адреса, застройщики и др.), которая используется в дальнейшей обработке и анализе.

In [ ]:
pip install requests beautifulsoup4 fake-useragent lxml

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import logging
from fake_useragent import UserAgent

Сначала парсим главную страницу с предложенными объявлениями и смотрим где находятся ссылки, которые нам надо вычленить

In [ ]:
url = 'https://move.ru/kvartiry_v_novostroykah/v_predelah_mkad/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

вычленяем эти самые ссылки с 400 страниц (в итоге вышло меньше, поскольку я по несколько раз продолжала парсить). На каждой странице по 30 ссылок

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import random


#Настройка браузера
def setup_driver():
    print("Настраиваем браузер")

    chrome_options = Options()  # Создание объекта для настройки опций Chrome
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')  # Скрытие автоматизации
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])  # Отключение опций автоматизации

    # Автоматическая установка драйвера
    service = Service(ChromeDriverManager().install())  # Создание службы с автоматической установкой ChromeDriver
    driver = webdriver.Chrome(service=service, options=chrome_options)  # Создание экземпляра браузера Chrome

    return driver

def main():
    driver = None  # Инициализация переменной драйвера (на случай ошибки)
    try:
        print("Запуск парсинга")
        driver = setup_driver()

        base_url = 'https://move.ru/kvartiry_v_novostroykah/v_predelah_mkad/'
        all_urls = []

        for page_num in range(1, 400):
            print(f"Страница {page_num}")

            # Формируем URL
            url = base_url if page_num == 1 else f"{base_url}?page={page_num}"  # Для первой страницы без параметра page
            print(f"Загружаем: {url}")

            # Переходим на страницу
            driver.get(url)  # Команда браузеру перейти по указанному URL
            time.sleep(3)

            links = driver.find_elements(By.XPATH, "//a[contains(@href, '/objects/')]")  # Поиск всех ссылок содержащих '/objects/'

            # Собираем URL
            new_urls = []
            for link in links:
                href = link.get_attribute('href')
                if href and href not in all_urls and href not in new_urls:
                    new_urls.append(href)

            all_urls.extend(new_urls)  # Добавление всех новых URL в общий список
            print(f"На странице {page_num} найдено {len(new_urls)} ссылок")


            wait_time = random.uniform(2, 4)
            print(f"Ждем {wait_time:.1f} секунд")
            time.sleep(wait_time)


        print(f"Сохраняем {len(all_urls)} URL")
        with open('extracted_urls.txt', 'w', encoding='utf-8') as f:
            for url in all_urls:
                f.write(url + '\n')

        print("Результаты в файле 'extracted_urls.txt'")

    except Exception as e:
        print(f"Ошибка: {e}")

    finally:  # Блок который выполнится в любом случае
        if driver:  # Проверка что драйвер существует
            driver.quit()  # Закрытие браузера
            print("Браузер закрыт")

if name == "main":
    main()

Открываем сохраненный файл с ссылками

In [ ]:
with open('extracted_urls.txt', 'r', encoding='utf-8') as file:
    urls = [line.strip() for line in file if line.strip()]

Найдено 10496 ссылок для парсинга


Теперь парсим одно из объявлений и смотрим его код, где находятся признаки, которые мы хотимм

In [ ]:
url = 'https://move.ru/objects/prodaetsya_2-komnatnaya_kvartira_ploschadyu_604_kvm_moskva_mojayskiy_rayon_ulica_vereyskaya_d_29s35_9285880157/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

In [ ]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import random


#Настройка браузера Chrome для парсинга
def setup_driver():
    print("Настраиваем браузер")

    chrome_options = Options() # Создаем объект для настроек Chrome
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')  # Скрытие автоматизации
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])  # Отключение опций автоматизации

    # Автоматическая установка драйвера
    service = Service(ChromeDriverManager().install())  # Создание службы с автоматической установкой ChromeDriver
    driver = webdriver.Chrome(service=service, options=chrome_options)  # Создание экземпляра браузера Chrome

    return driver

regex_patterns = {
    'id': re.compile(r'_(\d+)/?$'),  # _ затем одна/несколько цифр, затем необязательный / и конец строки $
    'price': re.compile(r'[\d\s\xa0]+'), # один/несколько символов из класса: цифры, пробелы, неразрывные пробелы
    'year': re.compile(r'(\d{4})'), #точно 4 цифры подряд
    'rooms': re.compile(r'(\d+)'), # одна/несколько цифр
    'area': re.compile(r'(\d+(?:\.\d+)?)'), # цифры, затем необязательная незахватываемая группа (?:) с точкой и цифрами
    'floor_current': re.compile(r'(\d+)/\d+'), # цифры, затем /, затем цифры
    'floor_total': re.compile(r'/(\d+)'), # символ /, затем цифры
    'views': re.compile(r'(\d+)'), # одна/несколько цифр
    'distance_km': re.compile(r'([\d,]+)\s*км'), # цифры/запятые, затем пробелы, затем текст км
    'distance_m': re.compile(r'(\d+)\s*м') # цифры, затем пробелы, затем текст м
}

def parse_property_page(driver, url, i, urls):
    try:
        if i % 5 == 0:
            print(f"Парсим {i}/{len(urls)}")

        driver.get(url)

        # Ждем загрузки основных элементов страницы (максимум 10 секунд)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'lxml')

        data = {}

        # 1. Извлекаем ID из URL
        id_match = regex_patterns['id'].search(url)
        data['id'] = id_match.group(1) if id_match else None

        # 2. Парсим цену
        try:
            price_elem = driver.find_element(By.CLASS_NAME, 'card-objects-price__main-price')
            price_text = price_elem.text.strip()

            # Очищаем текст цены от лишних символов
            clean_price = re.sub(r'[^\d,]', '', price_text.replace('\xa0', ''))
            clean_price = clean_price.replace(',', '.')

            if clean_price:
                price = float(clean_price)
                data['price_millions'] = round(price / 1000000, 2)
        except:
            data['price_millions'] = None
# 3-5. Парсим основную информацию: название ЖК, застройщик, класс жилья
        try:
            spec_items = driver.find_elements(By.CLASS_NAME, 'card-specifications-table__item')
            count = 0

            for item in spec_items:
                # Если нашли все три характеристики, прерываем цикл
                if count == 3:
                    break

                try:
                    item_text = item.text

                    # Название ЖК
                    if 'Название ЖК' in item_text:
                        try:
                            link_elem = item.find_element(By.CLASS_NAME, 'card-specifications-table__link')
                            data['complex_name'] = link_elem.text.strip()
                            count += 1
                        except:
                            pass

                    # Застройщик
                    elif 'Застройщик' in item_text:
                        try:
                            link_elem = item.find_element(By.CLASS_NAME, 'card-specifications-table__link')
                            data['developer'] = link_elem.text.strip()
                            count += 1
                        except:
                            pass

                    # Класс жилья
                    elif 'Класс жилья' in item_text:
                        try:
                            title_elem = item.find_element(By.CLASS_NAME, 'card-specifications-table__title')
                            data['housing_class'] = title_elem.text.strip()
                            count += 1
                        except:
                            pass

                except:
                    continue

        except:
            pass

        # 6-12. Парсим технические характеристики: площади, этажи, комнаты
        try:
            table_items = driver.find_elements(By.CLASS_NAME, 'card-specifications-table__item')
            count = 0

            for item in table_items:
                # Если нашли все 5 характеристик, прерываем цикл
                if count == 5:
                    break

                try:
                    item_text = item.text

                    # Общая площадь
                    if 'Общая площадь' in item_text and 'total_area' not in data:
                        match = regex_patterns['area'].search(item_text)
                        if match:
                            data['total_area'] = float(match.group(1))
                            count += 1

                    # Жилая площадь
                    elif 'Жилая площадь' in item_text and 'living_area' not in data:
                        match = regex_patterns['area'].search(item_text)
                        if match:
                            data['living_area'] = float(match.group(1))
                            count += 1

                    # Площадь кухни
                    elif 'Площадь кухни' in item_text and 'kitchen_area' not in data:
                        match = regex_patterns['area'].search(item_text)
                        if match:
                            data['kitchen_area'] = float(match.group(1))
                            count += 1

                    # Этаж
                    elif 'Этаж' in item_text and 'floor' not in data:
                        match = regex_patterns['floor_current'].search(item_text)
                        if match:
                            data['floor'] = int(match.group(1))

                        match = regex_patterns['floor_total'].search(item_text)
                        if match:
                            data['total_floors'] = int(match.group(1))

                        count += 1

                    # Количество комнат
elif 'Количество комнат' in item_text and 'rooms' not in data:
                        match = regex_patterns['rooms'].search(item_text)
                        if match:
                            data['rooms'] = int(match.group(1))
                            count += 1

                except:
                    continue

        except:
            pass

        # 13. Парсим полный адрес
        try:
            address_link = driver.find_element(By.CLASS_NAME, 'base-link.card-objects-location__address-link')
            data['full_address'] = address_link.get_attribute('title') or address_link.text.strip()
        except:
            data['full_address'] = None

        # 14-17. Парсим информацию о метро
        try:
            metro_stations = driver.find_elements(By.CLASS_NAME, 'card-objects-near-stations__station')
            metro_data = []

            for station in metro_stations:
                try:
                    # Название станции метро
                    name_elem = station.find_element(By.CLASS_NAME, 'card-objects-near-stations__station-link')
                    metro_name = name_elem.text.strip()

                    # Расстояние до метро
                    distance_m = None
                    try:
                        distance_elem = station.find_element(By.CLASS_NAME, 'card-objects-near-stations__station-distance')
                        distance_text = distance_elem.text.strip()

                        # Парсим километры
                        km_match = regex_patterns['distance_km'].search(distance_text)
                        if km_match:
                            distance_m = int(float(km_match.group(1).replace(',', '.')) * 1000)
                        else:
                            # Парсим метры
                            m_match = regex_patterns['distance_m'].search(distance_text)
                            if m_match:
                                distance_m = int(m_match.group(1))

                    except:
                        pass

                    metro_data.append({
                        'name': metro_name,
                        'distance_m': distance_m
                    })

                except:
                    continue

            # Сортируем станции метро по расстоянию и берем ближайшую
            if metro_data:
                metro_data.sort(key=lambda x: x['distance_m'] if x['distance_m'] else float('inf'))
                nearest = metro_data[0]

                data['metro_data'] = metro_data
                data['metro_names'] = [m['name'] for m in metro_data]
                data['nearest_metro'] = nearest['name']
                data['nearest_metro_distance'] = nearest['distance_m']

        except:
            pass

        # 18. Парсим количество просмотров
        try:
            views_elems = driver.find_elements(By.CLASS_NAME, 'card-meta__item')
            for elem in views_elems:
                views_text = elem.text.strip()
                if 'просмотр' in views_text.lower():
                    views_match = regex_patterns['views'].search(views_text)
                    if views_match:
                        data['views'] = int(views_match.group(1))
                        break
        except:
            pass

        data['url'] = url

        return data

    except Exception as e:
        print(f"Ошибка при парсинге {url}: {e}")
        return None
def main():
    driver = None
    all_data = []

    try:
        # Настраиваем браузер
        driver = setup_driver()
        print("Браузер запущен, начинаем парсинг")

        # Читаем URL из файла
        with open('extracted_urls.txt', 'r', encoding='utf-8') as f:
            urls = [line.strip() for line in f if line.strip()]

        print(f"Найдено {len(urls)} URL для парсинга")

        # Парсим каждое объявление
        for i, url in enumerate(urls, 1):
            data = parse_property_page(driver, url, i, urls)
            if data:
                all_data.append(data)

            time.sleep(random.uniform(0.5, 1.5))

            # Дополнительная пауза каждые 20 объявлений
            if i % 20 == 0:
                sleep_time = random.uniform(2, 4)
                print(f"Пауза {sleep_time:.1f} секунд")
                time.sleep(sleep_time)

    except Exception as e:
        print(f"Критическая ошибка: {e}")

    finally:
        # Закрываем браузер в любом случае
        if driver:
            driver.quit()
            print("Браузер закрыт")

    # Создаем DataFrame и сохраняем результаты
    if all_data:
        df = pd.DataFrame(all_data)
        print(f"\nУспешно собрано {len(df)} объявлений")

        df.to_csv('parsingMOVE.csv', index=False, encoding='utf-8')
        print("Данные сохранены в parsingMOVE.csv")

    else:
        print("Не удалось собрать данные")

if name == "main":
    main()